In [1]:
import time
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
%matplotlib qt
import math
import cv2

In [3]:
'''
%VGG_KR_FROM_P Extract K, R from camera matrix.
%
%    [K,R,t] = VGG_KR_FROM_P(P [,noscale]) finds K, R, t such that P = K*R*[eye(3) -t].
%    It is det(R)==1.
%    K is scaled so that K(3,3)==1 and K(1,1)>0. Optional parameter noscale prevents this.
%
%    Works also generally for any P of size N-by-(N+1).
%    Works also for P of size N-by-N, then t is not computed.

% original Author: Andrew Fitzgibbon <awf@robots.ox.ac.uk> and awf
% Date: 15 May 98

% Modified by Shu, ANU.
% Date: 8 May 20
'''

def vgg_rq(S):
    S = S.T
    [Q,U] = np.linalg.qr(S[::-1,::-1], mode='complete')

    Q = Q.T
    Q = Q[::-1, ::-1]
    U = U.T
    U = U[::-1, ::-1]
    if np.linalg.det(Q)<0:
        U[:,0] = -U[:,0]
        Q[0,:] = -Q[0,:]
    return U,Q


def vgg_KR_from_P(P, noscale = True):
    N = P.shape[0]
    H = P[:,0:N]
#     print(N,'|', H)
    [K,R] = vgg_rq(H)
    if noscale:
        K = K / K[N-1,N-1]
        if K[0,0] < 0:
            D = np.diag([-1, -1, np.ones([1,N-2])]);
            K = K @ D
            R = D @ R
        
            test = K*R; 
            assert (test/test[0,0] - H/H[0,0]).all() <= 1e-07
    
    t = np.linalg.inv(-P[:,0:N]) @ P[:,-1]
    return K, R, t

# K, R, t = vgg_KR_from_P(C)
# print()

In [4]:


############################################################################
def rq(A):
    # RQ factorisation

    [q,r] = np.linalg.qr(A.T)   # numpy has QR decomposition, here we can do it 
                                # with Q: orthonormal and R: upper triangle. Apply QR
                                # for the A-transpose, then A = (qr).T = r.T@q.T = RQ
    R = r.T
    Q = q.T
    return R,Q



In [4]:
# Task 1

In [5]:
# I = Image.open('stereo2012a.jpg')
# plt.title('select 12 points in the image')
# plt.imshow(I)

# uv = np.asarray(plt.ginput(12,timeout=9999)) # Graphical user interface to get 12 points
# np.save('uv1.npy',uv)


In [6]:

XYZ = np.array([
    [0,35,21],
    [0,28,14],
    [0,21,7],
    [0,7,14],
    [21,35,0],
    [14,28,0],
    [7,21,0],
    [14,7,0],
    [0,0,0],
    [7,0,21],
    [21,0,21],
    [21,0,7]
])


In [7]:

img = cv2.imread('stereo2012a.jpg')
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

uv1 = np.load('uv1.npy')

uv_3D = (np.append(uv1,np.ones((12,1)),axis=1))
XYZ_4D = (np.append(XYZ,np.ones((12,1)),axis=1))

# plt.imshow(img)


In [8]:

plt.figure()

plt.scatter(uv1[:,0],uv1[:,1],c='r',s=30,marker='o',zorder=1)
plt.title('Check the selected points in the image')
plt.imshow(img)


In [9]:

'''
%% TASK 1: CALIBRATE
%
% Function to perform camera calibration
%
% Usage:   calibrate(image, XYZ, uv)
%          return C
%   Where:   image - is the image of the calibration target.
%            XYZ - is a N x 3 array of  XYZ coordinates
%                  of the calibration target points. 
%            uv  - is a N x 2 array of the image coordinates
%                  of the calibration target points.
%            K   - is the 3 x 4 camera calibration matrix.
%  The variable N should be an integer greater than or equal to 6.
%
%  This function plots the uv coordinates onto the image of the calibration
%  target. 
%
%  It also projects the XYZ coordinates back into image coordinates using
%  the calibration matrix and plots these points too as 
%  a visual check on the accuracy of the calibration process.
%
%  Lines from the origin to the vanishing points in the X, Y and Z
%  directions are overlaid on the image. 
%
%  The mean squared error between the positions of the uv coordinates 
%  and the projected XYZ coordinates is also reported.
%
%  The function should also report the error in satisfying the 
%  camera calibration matrix constraints.
% 
% your name: Runxiang Huang
% date: 12 May
'''

#####################################################################

def T_norm(img):
    img_copy = img.copy()
    H = img_copy.shape[0]
    W = img_copy.shape[1]
    matrix = np.array([
        [H + W, 0, W / 2],
        [0, H + W, H / 2],
        [0, 0, 1]
    ])
    T_norm = np.linalg.inv(matrix)
    return T_norm

def S_norm(uv):
# calculate the coovariance matrix of uv
    Cov = np.cov(uv.T)
# generate the eigenvalues and eigenvectors by the Cov
    EigenValues, EigenVectors = np.linalg.eig(Cov)
# generate the diagonal matrix with the eigenvalues
    Diag = np.zeros((EigenValues.shape[0],EigenValues.shape[0]))
    for i in range(EigenValues.shape[0]):
        if EigenValues[i] != 0:
            Diag[i][i] = 1 / EigenValues[i]
        else:
            Diag[i][i] = EigenValues[i]
# generate the S_norm matrix according to the formulas
    S_norm = np.zeros((EigenValues.shape[0] + 1, EigenVectors.shape[1] + 1))
    S_norm[:EigenVectors.shape[0], :EigenVectors.shape[1]] = EigenVectors
    S_norm[:EigenVectors.shape[0], - 1] = EigenValues
    S_norm[-1, -1] = 1
    return S_norm


def generate_x(img, XYZ, uv):
    t_norm = T_norm(img)
    s_norm = S_norm(uv)
    x_i =  uv @ t_norm.T # 12*3
    X_i = XYZ @ s_norm.T # 12*4
    return X_i, x_i

def calibrate(img, XYZ, uv):
    uv_3D = (np.append(uv, np.ones((uv.shape[0], 1)), axis=1))
    XYZ_4D = (np.append(XYZ, np.ones((XYZ.shape[0], 1)), axis=1))
    X_i, x_i = generate_x(img, XYZ_4D, uv_3D)
    i_range = uv.shape[0]
    A = []
# according to the formulas, generate the A matrix
    for i in range(i_range):
        x = list(X_i[i][:])
        u = x_i[i][0]
        v = x_i[i][1]
        
        ux = [a*-u for a in x]
        vx = [b*-v for b in x]
        
        A.append(np.array(x + [0, 0, 0, 0] + ux))
        A.append(np.array([0, 0, 0, 0] + x + vx))

    A = np.stack(A)
    s, v, d = np.linalg.svd(A)
    P = d.T[:, -1].reshape(3, 4)
    return P

def denormalize_DLT(img, P, uv):
    uv_3D = (np.append(uv,np.ones((uv.shape[0],1)),axis=1))
    t_norm = T_norm(img)
    s_norm = S_norm(uv_3D)
    C = np.linalg.inv(t_norm) @ P @ s_norm
    return C


In [10]:

P = calibrate(img, XYZ, uv1)
C = denormalize_DLT(img, P, uv1)
print(C)


[[ 4.30692235e-02 -2.01398217e-02 -5.80933677e-02  3.13168777e+00]
 [ 1.80922450e-03 -7.24391482e-02  1.40454359e-02  3.24920600e+00]
 [-3.61910697e-05 -3.40535021e-05 -5.15006470e-05  9.70806299e-03]]


In [11]:

new_xyz = XYZ_4D @ C.T

new_points = np.zeros((12,2))
new_points[:,0] = new_xyz[:,0]/new_xyz[:,2]
new_points[:,1] = new_xyz[:,1]/new_xyz[:,2]

error = np.sqrt(np.mean((uv1 - new_points) * (uv1 - new_points)))
print('RMSE is ' + str(error))

plt.figure()

plt.scatter(uv_3D[:,0],uv_3D[:,1],c='r',s=30,marker='o',zorder=1)
plt.scatter(new_points[:,0],new_points[:,1],c='y',s=25,marker='+',zorder=2)
plt.title('Show the points with normalized DLT')

plt.imshow(img)


RMSE is 1.0235408220566287


In [12]:

K,R,t = vgg_KR_from_P(C)
print("K matrix is \n", K)
print("R matrix is \n", R)
print("t matrix is \n", t)

error = np.sqrt(np.mean((uv1 - new_points) * (uv1 - new_points)))
print('RMSE is ' + str(error))

alpha = K[0][0]
gamma = K[0][1]
beta = K[1][1]

fx = alpha
theta = math.atan(-alpha / gamma)
fy = beta * math.sin(theta)
f = np.sqrt(fx**2 + fy**2)

print('Focal length is ' + str(f))

Pitch_angle = math.asin(-R[2][0])
print("pitch angle =",math.degrees(Pitch_angle))


K matrix is 
 [[963.89803202   5.32622531 413.7171333 ]
 [  0.         977.93687612 327.61839117]
 [  0.           0.           1.        ]]
R matrix is 
 [[ 0.8403214  -0.08288359 -0.53571471]
 [ 0.19526484 -0.87562285  0.44176496]
 [-0.5056991  -0.47583079 -0.71962037]]
t matrix is 
 [79.18875324 64.3426329  90.31042518]
RMSE is 1.0235408220566287
Focal length is 1373.109882577227
pitch angle = 30.377771285939488


In [13]:

img_copy = img.copy()

Height = img.shape[0]
Width = img.shape[1]

Half_H = Height // 2
Half_W = Width // 2
resized_img = cv2.resize(img_copy, (Half_W, Half_H), interpolation = cv2.INTER_AREA)

plt.imshow(resized_img)


In [14]:

# plt.imshow(resized_img)
# plt.title('resized image')

# resized_uv = np.asarray(plt.ginput(12,timeout=9999))
# np.save('resized_uv.npy',resized_uv)


In [15]:

resized_uv = np.load('resized_uv.npy')
resized_xyz_4D = XYZ_4D
resized_xyz = XYZ
resized_uv_3D = (np.append(resized_uv,np.ones((12,1)),axis=1))


In [16]:

resized_P = calibrate(resized_img, resized_xyz, resized_uv)
resized_C = denormalize_DLT(resized_img, resized_P, resized_uv)
print(resized_C)



[[-7.85955179e-02  3.98700409e-02  1.21333988e-01 -6.40764504e+00]
 [ 6.22101874e-03  1.44861802e-01 -2.26415381e-02 -6.69252025e+00]
 [ 1.96403841e-04  1.30334756e-04  2.30874007e-04 -3.98206267e-02]]


In [17]:

plt.figure()

plt.scatter(resized_uv_3D[:,0], resized_uv_3D[:,1], c = 'r', s = 30, marker = 'o', zorder = 1)
plt.title('Check the selected points in the resized image')
plt.imshow(resized_img)



In [18]:

new_resized_xyz = resized_xyz_4D @ resized_C.T

new_resized_points = np.zeros((12,2))
new_resized_points[:,0] = new_resized_xyz[:,0] / new_resized_xyz[:,2]
new_resized_points[:,1] = new_resized_xyz[:,1] / new_resized_xyz[:,2]

plt.figure()

plt.scatter(resized_uv_3D[:,0], resized_uv_3D[:,1], c = 'r', s = 30, marker = 'o', zorder = 1)
plt.scatter(new_resized_points[:,0], new_resized_points[:,1], c = 'y', s = 25, marker = '+', zorder = 2)
plt.title('Show the resized image points with normalized DLT')

plt.imshow(resized_img)



In [19]:

resized_error = np.sqrt(np.mean((resized_uv - new_resized_points) * (resized_uv - new_resized_points)))
print('Resized RMSE is ' + str(resized_error))

resized_K, resized_R, resized_t = vgg_KR_from_P(resized_C)
print('Resized K matrix is \n' + str(resized_K))
print('Resized R matrix is \n' + str(resized_R))
print('Resized t matrix is \n' + str(resized_t))


Resized RMSE is 0.6388344535777464
Resized K matrix is 
[[424.16711024   2.41035104 163.25677073]
 [ -0.         423.26904508 136.63799726]
 [ -0.          -0.           1.        ]]
Resized R matrix is 
[[ 0.78985765 -0.1276756  -0.59985318]
 [ 0.14761423 -0.90975791  0.38800848]
 [-0.59526039 -0.39501833 -0.69973251]]
Resized t matrix is 
[71.08514849 55.73511028 80.5418016 ]


In [20]:

print('Resized K matrix is \n' + str(resized_K))
print("K matrix is \n" + str(K))
print('Resized R matrix is \n' + str(resized_R))
print("R matrix is \n" + str(R))
print('Resized t matrix is \n' + str(resized_t))
print("t matrix is \n" + str(t))


Resized K matrix is 
[[424.16711024   2.41035104 163.25677073]
 [ -0.         423.26904508 136.63799726]
 [ -0.          -0.           1.        ]]
K matrix is 
[[963.89803202   5.32622531 413.7171333 ]
 [  0.         977.93687612 327.61839117]
 [  0.           0.           1.        ]]
Resized R matrix is 
[[ 0.78985765 -0.1276756  -0.59985318]
 [ 0.14761423 -0.90975791  0.38800848]
 [-0.59526039 -0.39501833 -0.69973251]]
R matrix is 
[[ 0.8403214  -0.08288359 -0.53571471]
 [ 0.19526484 -0.87562285  0.44176496]
 [-0.5056991  -0.47583079 -0.71962037]]
Resized t matrix is 
[71.08514849 55.73511028 80.5418016 ]
t matrix is 
[79.18875324 64.3426329  90.31042518]


In [21]:
# Task 2

In [22]:

# I1 = Image.open('left.jpg')
# plt.title('select 6 points in the origin image')
# plt.imshow(I1)

# base = np.asarray(plt.ginput(6,timeout=9999))
# np.save('base.npy', base)


In [23]:

# I2 = Image.open('right.jpg')
# plt.title('select 6 coresponding points in the trans image')
# plt.imshow(I2)

# trans = np.asarray(plt.ginput(6,timeout=9999))
# np.save('trans.npy', trans)



In [5]:

img2 = cv2.imread('left.jpg')
img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2RGB)
img3 = cv2.imread('right.jpg')
img3 = cv2.cvtColor(img3, cv2.COLOR_BGR2RGB)


In [6]:

base = np.load('base.npy')
plt.figure()

plt.scatter(base[:,0],base[:,1],c='r',s=30,marker='o',zorder=1)
plt.title('Check the selected points in the image')
plt.imshow(img2)



In [7]:

trans = np.load('trans.npy')
plt.figure()

plt.scatter(trans[:,0],trans[:,1],c='r',s=30,marker='o',zorder=1)
plt.title('Check the coresponding points in the image')
plt.imshow(img3)



In [8]:

plt.figure()
plt.subplot(121)
plt.scatter(base[:,0],base[:,1],c='r',s=30,marker='o',zorder=1)
plt.title('The selected points in the left image')
plt.imshow(img2)

plt.subplot(122)
plt.scatter(trans[:,0],trans[:,1],c='r',s=30,marker='o',zorder=1)
plt.title('Check the corresponding points in right the image')
plt.imshow(img3)


In [28]:

# plt.figure()
# plt.imshow(img2)
# plt.title('select 6 points from the image')
# uv2 = np.asarray(plt.ginput(6,timeout=9999)) # Graphical user interface to get 6 points

# np.save('uv2.npy', uv2)


In [9]:

uv2 = np.load('uv2.npy')
plt.figure()

plt.scatter(uv2[:,0],uv2[:,1],c='r',s=30,marker='o',zorder=1)
plt.title('Check the selected points in the image')
plt.imshow(img2)


In [10]:
############################################################################


'''
%% TASK 2: 
% Computes the homography H applying the Direct Linear Transformation 
% The transformation is such that 
% p = np.matmul(H, p.T), i.e.,
% (uBase, vBase, 1).T = np.matmul(H, (u2Trans , v2Trans, 1).T)
% Note: we assume (a, b, c) => np.concatenate((a, b, c), axis), be careful when 
% deal the value of axis 
%
% INPUTS: 
% u2Trans, v2Trans - vectors with coordinates u and v of the transformed image point (p') 
% uBase, vBase - vectors with coordinates u and v of the original base image point p  
% 
% OUTPUT 
% H - a 3x3 Homography matrix  
% 
% your name, date 
'''

def T_norm(img):
    img_copy = img.copy()
    H = img_copy.shape[0]
    W = img_copy.shape[1]
    matrix = np.array([
        [H + W, 0, W/2],
        [0, H + W, H/2],
        [0, 0, 1]
    ])
    T_norm = np.linalg.inv(matrix)
    return T_norm

def homography(u2Trans, v2Trans, uBase, vBase):
    A = []
    for i in range(len(u2Trans)):
        x = uBase[i]
        y = vBase[i]
        u = u2Trans[i]
        v = v2Trans[i]
        
        
        A.append(np.array([0, 0, 0, -x, -y, -1, v * x, v * y, v]))
        A.append(np.array([x, y, 1, 0, 0, 0, -u * x, -u * y, -u]))
        
    A = np.stack(A)
    s, v, d = np.linalg.svd(A)
    H = d[-1,:].reshape(3, 3)
#     H = H/H[-1,-1]
    return H

def denormalize_H(T_norm2, H, T_norm1):
    new_H = np.linalg.inv(T_norm2) @ H @ T_norm1
    return new_H


In [11]:

base_3D = (np.append(base,np.ones((6,1)),axis=1))
trans_3D = (np.append(trans,np.ones((6,1)),axis=1))
uv2_3D = (np.append(uv2,np.ones((6,1)),axis=1))

t_norm_1 = T_norm(img2)
t_norm_2 = T_norm(img3)

normalized_base = base_3D @ t_norm_1.T
normalized_trans = trans_3D @ t_norm_2.T

H = homography(normalized_trans[:,0], normalized_trans[:,1], normalized_base[:,0], normalized_base[:,1])
H = denormalize_H(t_norm_2, H, t_norm_1)
print(H)


[[-6.69453868e-01  1.70167581e-02  4.93198796e+01]
 [-1.17573027e-01 -2.85337692e-01  5.13632082e+00]
 [-8.68212535e-04  4.25342028e-05 -1.71512667e-01]]


In [32]:

hz = (H @ uv2_3D.T)

new = np.zeros((2,6))
new[0] = hz[0]/hz[2]
new[1] = hz[1]/hz[2]

plt.figure()
plt.subplot(122)
plt.imshow(img3)
plt.title('Show the normalized homography points')
plt.scatter(new[0],new[1],c='r',s=30,marker='o',zorder=2)


plt.subplot(121)
plt.scatter(uv2[:,0],uv2[:,1],c='r',s=30,marker='o',zorder=1)
plt.title('Check the selected points in the image')
plt.imshow(img2)


In [33]:
plt.figure()
warp_img = cv2.warpPerspective(img2, H, (img2.shape[1],img2.shape[0]))
plt.imshow(warp_img)
plt.title('Show the wrapping image result')
plt.show()

In [34]:
# test the distance of the selected points affect the result on the warpping image

In [35]:

# plt.figure()
# plt.imshow(img2)
# test_base_uv = np.asarray(plt.ginput(6,timeout=9999)) # Graphical user interface to get 6 points

# np.save('test_base_uv.npy', test_base_uv)


In [36]:

# plt.figure()
# plt.imshow(img3)
# test_trans_uv = np.asarray(plt.ginput(6,timeout=9999)) # Graphical user interface to get 6 points

# np.save('test_trans_uv.npy', test_trans_uv)


In [37]:

test_base_uv = np.load('test_base_uv.npy')
test_trans_uv = np.load('test_trans_uv.npy')

test_base_3D = (np.append(test_base_uv,np.ones((6,1)),axis=1))
test_trans_3D = (np.append(test_trans_uv,np.ones((6,1)),axis=1))
uv2_3D = (np.append(uv2,np.ones((6,1)),axis=1))

t_norm_1 = T_norm(img2)
t_norm_2 = T_norm(img3)

norm_test_base = test_base_3D @ t_norm_1.T
norm_test_trans = test_trans_3D @ t_norm_2.T

test_H = homography(norm_test_trans[:,0], norm_test_trans[:,1], norm_test_base[:,0], norm_test_base[:,1])
test_H = denormalize_H(t_norm_2, test_H, t_norm_1)
print(test_H)




[[ 5.45246838e-01  1.67508782e-01 -9.72470917e+01]
 [ 1.35145131e-01  3.39167434e-01 -5.03698733e+01]
 [ 8.57674857e-04  5.58742974e-04 -8.53183820e-02]]


In [38]:

new_hz = (test_H @ uv2_3D.T)

test_new = np.zeros((2,6))
test_new[0] = new_hz[0]/new_hz[2]
test_new[1] = new_hz[1]/new_hz[2]

# plt.figure()
# plt.subplot(122)
# plt.imshow(img3)
# plt.title('Show the normalized homography points')
# plt.scatter(test_new[0],test_new[1],c='r',s=30,marker='o',zorder=2)


# plt.subplot(121)
# plt.scatter(uv2[:,0],uv2[:,1],c='r',s=30,marker='o',zorder=1)
# plt.title('Check the selected points in the image')
# plt.imshow(img2)
plt.figure()
plt.scatter(test_base_uv[:,0],test_base_uv[:,1],c='r',s=30,marker='o',zorder=1)
plt.imshow(img2)



In [39]:
plt.figure()
test_warp_img = cv2.warpPerspective(img2, test_H, (img2.shape[1],img2.shape[0]))
plt.imshow(test_warp_img)
plt.title('Show the test1 wrapping image result')
plt.show()

In [40]:

# plt.figure()
# plt.imshow(img2)
# test2_base_uv = np.asarray(plt.ginput(6,timeout=9999)) # Graphical user interface to get 6 points

# np.save('test2_base_uv.npy', test2_base_uv)


# plt.figure()
# plt.imshow(img3)
# test2_trans_uv = np.asarray(plt.ginput(6,timeout=9999)) # Graphical user interface to get 6 points

# np.save('test2_trans_uv.npy', test2_trans_uv)



In [41]:

test2_base_uv = np.load('test2_base_uv.npy')
test2_trans_uv = np.load('test2_trans_uv.npy')

test2_base_3D = (np.append(test2_base_uv,np.ones((6,1)),axis=1))
test2_trans_3D = (np.append(test2_trans_uv,np.ones((6,1)),axis=1))
uv2_3D = (np.append(uv2,np.ones((6,1)),axis=1))

t_norm_1 = T_norm(img2)
t_norm_2 = T_norm(img3)

norm_test2_base = test2_base_3D @ t_norm_1.T
norm_test2_trans = test2_trans_3D @ t_norm_2.T

test2_H = homography(norm_test2_trans[:,0], norm_test2_trans[:,1], norm_test2_base[:,0], norm_test2_base[:,1])
test2_H = denormalize_H(t_norm_2, test2_H, t_norm_1)
print(test_H)


plt.figure()
test2_warp_img = cv2.warpPerspective(img2, test2_H, (img2.shape[1],img2.shape[0]))
plt.imshow(test2_warp_img)
plt.title('Show the test2 wrapping image result')
plt.show()


[[ 5.45246838e-01  1.67508782e-01 -9.72470917e+01]
 [ 1.35145131e-01  3.39167434e-01 -5.03698733e+01]
 [ 8.57674857e-04  5.58742974e-04 -8.53183820e-02]]


In [42]:
plt.figure()
plt.scatter(test2_base_uv[:,0],test2_base_uv[:,1],c='r',s=30,marker='o',zorder=1)
plt.imshow(img2)